In [ ]:
# This is the consequence processor for sgRNA designed by CHOP CHOP
# This notebook will process the local chopchop output and is tied to the source function sgFinder
# Author: Yiyun
# Date: Sep, 2022

In [1]:
from os import listdir
from os.path import join
import itertools
import pandas as pd 
from tqdm import tqdm
import pickle
from Bio import SeqIO

from os.path import exists
from os import listdir,makedirs

import sys
sys.path.append('../')
from src import syn_sgFinder

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
out_path = '../data/output/sg_Finder'
mrna_path = '../data/MANE'
fpath = '../data/output/sg_out/'

# Load all expressed transcripts
f_genelist = pd.read_csv('../data/output/k562_comm_ess.txt', header =None)
list_exp_trsp = pickle.load(open('../data/22Q2_gene_effect/expressed_transcripts.pkl','rb'))
# Parse mRNA sequence dictionary, for use when sequence aligned to intron-exon junctions
record_dict = SeqIO.to_dict(SeqIO.parse(join(mrna_path,'GRCh38_latest_rna.fna'), "fasta"))

In [52]:
def run_sgFinder(input_genes, tag = 'ess', sg_p ='../data/output/sg_out' ):
    out_path = '../data/output/sg_Finder'
    
    df_synsg_abe = pd.DataFrame()
    df_synsg_cbe = pd.DataFrame()
    df_abeCT = pd.DataFrame()
    df_cbeCT = pd.DataFrame()

    for genes in tqdm(input_genes):
        sg_gene = syn_sgFinder.sgFinder_gene(genes, sg_path = sg_p)
        try:
            sg_gene.gene_cds_proc()
        except ValueError as e:
            print(e)
            continue

        df_synsg_abe_dt = pd.DataFrame()
        df_synsg_cbe_dt = pd.DataFrame()
        for idx, transcripts in list(itertools.product(sg_gene.sg_file.index,\
                                                           sg_gene.dict_exons.keys())):
            if not transcripts in list_exp_trsp:
    #             print(f'{transcripts} not expressed, skipping...')
                continue # Pass the transcripts that are not expressed
            else:
                sg_sg = syn_sgFinder.sgFinder_sg(sg_gene.gene,idx)
                check_point = sg_sg.process_BE_window(sg_gene)
                if check_point:
                    sg_trsp = syn_sgFinder.sgFinder_transcript(sg_gene.gene,idx,transcripts)
                    sg_trsp.all_pos_consq(sg_gene,sg_sg,record_dict)

                    df_synsg_abe = df_synsg_abe.append(sg_trsp.df_abe,ignore_index = True)
                    df_synsg_cbe = df_synsg_cbe.append(sg_trsp.df_cbe,ignore_index = True)

                    df_abeCT = df_abeCT.append(sg_trsp.df_abe_control,ignore_index = True)
                    df_cbeCT = df_cbeCT.append(sg_trsp.df_cbe_control,ignore_index = True)

                    df_synsg_abe_dt = df_synsg_abe_dt.append(sg_trsp.df_abe_ind,ignore_index = True)
                    df_synsg_cbe_dt = df_synsg_cbe_dt.append(sg_trsp.df_cbe_ind,ignore_index = True)

        df_synsg_abe_dt.to_csv(join(out_path,genes,f'df_abe_detail.csv'))
        df_synsg_cbe_dt.to_csv(join(out_path,genes,f'df_cbe_detail.csv'))
    
    # Process and saving final result
    ABE_synsg = df_synsg_abe[df_synsg_abe['syn_or_not'].apply(lambda x: all(x)&any(x))]
    CBE_synsg = df_synsg_cbe[df_synsg_cbe['syn_or_not'].apply(lambda x: all(x)&any(x))]
    
    if not exists(join(out_path,tag)):
        makedirs(join(out_path,tag))
    ABE_synsg.to_csv(join(out_path,tag,'ABE_synsg.csv'))
    CBE_synsg.to_csv(join(out_path,tag,'CBE_synsg.csv'))

    df_abeCT.to_csv(join(out_path,tag,'ABE_CT.csv'))
    df_cbeCT.to_csv(join(out_path,tag,'CBE_CT.csv'))
    
    return(ABE_synsg)

***Run essential gene list***

In [54]:
fpath = '../data/output/sg_out/'
list_gene = [i for i in listdir(fpath) if not i.startswith('.')]
list_gene = [i for i in list_gene if len(listdir(join(fpath,i))) > 5]
list_gene = [i for i in list_gene if i in list(f_genelist[0])]
abe_df = run_sgFinder(list_gene, tag = 'ess' )

  0%|                                                   | 0/122 [00:00<?, ?it/s]

Processing CLP1...Keep 423/477 low off-target sgRNAs...
ATTACATCTCGTTTAGCAGA in gene CLP1 not aligned, skip...
Finding mrna sequence in another exon...
57661427 is in start/stop codon ...skippingCTACTTCAGAT
57661427 is in start/stop codon ...skippingCTACTTCAGAT
CTACTTCAGATCCATGAACC in gene CLP1 not aligned, skip...


  1%|▎                                          | 1/122 [00:03<06:45,  3.36s/it]

57659487 is in start/stop codon ...skippingATGGGAGAAGA
ATGGGAGAAGAGGCTAATGA in gene CLP1 not aligned, skip...
Processing RBBP4...Keep 4/354 low off-target sgRNAs...
Processing EIF4A3...Keep 371/443 low off-target sgRNAs...
CGTAAGCGTAGATGCCCCGC in gene EIF4A3 not aligned, skip...
80146951 is in start/stop codon ...skippingATGGCGACCAC
ATGGCGACCACGGCCACGAT in gene EIF4A3 not aligned, skip...
CGTTTCACCAAGATGCGGAT in gene EIF4A3 not aligned, skip...
Finding mrna sequence in another exon...
Finding extra sequence in mRNA failed, skip...
Finding mrna sequence in another exon...
Finding extra sequence in mRNA failed, skip...


  2%|█                                          | 3/122 [00:06<03:55,  1.98s/it]

...

Processing EIF3I...Keep 207/348 low off-target sgRNAs...
CAATCTGGCTCGGATCCCGC in gene EIF3I not aligned, skip...
Finding mrna sequence in another exon...
AGGGTCCTTGGCCACAGTAA in gene EIF3I not aligned, skip...
Finding mrna sequence in another exon...
32231186 is in start/stop codon ...skippingTTAAGCCTCAA


 99%|████████████████████████████████████████▋| 121/122 [13:17<00:16, 16.76s/it]

CTGTTTCACAGTCCCAGAGA in gene EIF3I not aligned, skip...
Processing CDT1...Keep 692/822 low off-target sgRNAs...
88803841 is in start/stop codon ...skippingATGGAGCAGCG
ATGGAGCAGCGCCGCGTCAC in gene CDT1 not aligned, skip...
CTGGTCCTGCGCGGATGTCA in gene CDT1 not aligned, skip...
Finding mrna sequence in another exon...
Finding mrna sequence in another exon...
Finding mrna sequence in another exon...
Finding mrna sequence in another exon...
88808267 is in start/stop codon ...skippingCACAGCCCCTC
88808267 is in start/stop codon ...skippingCACAGCCCCTC
CACAGCCCCTCCTCAGCACG in gene CDT1 need sequence not included in fasta file...


100%|█████████████████████████████████████████| 122/122 [13:27<00:00,  6.62s/it]

Finding mrna sequence in another exon...


***Run opposite + depletion gene list***

In [77]:
list_gene_enrich = [i for i in listdir('../data/output/sg_out_enrich') if not i.startswith('.')]
list_gene_enrich = [i for i in list_gene_enrich if len(listdir(join('../data/output/sg_out_enrich',i))) > 5]
run_sgFinder(list_gene_enrich, tag = 'enr' , sg_p ='../data/output/sg_out_enrich' )

  6%|██▌                                         | 1/17 [00:00<00:02,  5.99it/s]

Processing AMN...Keep 574/648 low off-target sgRNAs...
Processing NKX6-1...Keep 436/601 low off-target sgRNAs...


 12%|█████▏                                      | 2/17 [00:00<00:02,  6.27it/s]

Processing ZBTB20...Keep 905/1017 low off-target sgRNAs...


 24%|██████████▎                                 | 4/17 [00:13<00:48,  3.75s/it]

...

 82%|███████████████████████████████████▍       | 14/17 [00:32<00:03,  1.26s/it]

Processing AOAH...Keep 423/494 low off-target sgRNAs...


 94%|████████████████████████████████████████▍  | 16/17 [00:33<00:00,  1.35it/s]

Processing HOXD1...Keep 450/527 low off-target sgRNAs...
Processing C2CD4B...Keep 294/673 low off-target sgRNAs...


100%|███████████████████████████████████████████| 17/17 [00:33<00:00,  1.96s/it]


***For Debugging...***
***Single gene checks***

In [13]:
sg_gene = syn_sgFinder.sgFinder_gene('MARS1')
sg_gene.gene_cds_proc()
sg_sg = syn_sgFinder.sgFinder_sg(sg_gene.gene,2)
sg_sg.process_BE_window(sg_gene)
sg_trsp = syn_sgFinder.sgFinder_transcript(sg_gene.gene,2, 'NM_021960')
sg_trsp.all_pos_consq(sg_gene,sg_sg)

FileNotFoundError: [Errno 2] No such file or directory: '../data/output/sg_out_enrich/MARS1/MARS1.txt'